In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 7.6MB/s 


In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Work/Projects/Bank Leave Predictor Project/churn.csv')

In [5]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
X = data.iloc[:,3:-1]
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [7]:
encoder = OrdinalEncoder()
value = encoder.fit_transform(X['Geography'].values.reshape(-1, 1))
X['Geography'] = value
encoder = OrdinalEncoder()
value = encoder.fit_transform(X['Gender'].values.reshape(-1, 1))
X['Gender'] = value

In [8]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0.0,0.0,42,2,0.00,1,1,1,101348.88
1,608,2.0,0.0,41,1,83807.86,1,0,1,112542.58
2,502,0.0,0.0,42,8,159660.80,3,1,0,113931.57
3,699,0.0,0.0,39,1,0.00,2,0,0,93826.63
4,850,2.0,0.0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0.0,1.0,39,5,0.00,2,1,0,96270.64
9996,516,0.0,1.0,35,10,57369.61,1,1,1,101699.77
9997,709,0.0,0.0,36,7,0.00,1,0,1,42085.58
9998,772,1.0,1.0,42,3,75075.31,2,1,0,92888.52


In [9]:
y = data.iloc[:,len(data.columns)-1]
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [11]:
len(X_train), len(X_test), len(y_train), len(y_test)

(7000, 3000, 7000, 3000)

In [12]:
RF = RandomForestClassifier(n_estimators = 100, max_depth = 2, random_state = 0)
RF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [13]:
round(RF.score(X_train, y_train), 4)

0.8093

In [14]:
round(RF.score(X_test, y_test), 4)

0.8237

In [15]:
perm = PermutationImportance(RF, random_state = 42, n_iter = 10).fit(X, y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

Weight,Feature
0.0196 ± 0.0014,NumOfProducts
0.0070 ± 0.0011,Age
0.0012 ± 0.0008,Balance
0.0003 ± 0.0003,Geography
0.0002 ± 0.0006,Gender
0.0002 ± 0.0000,CreditScore
0.0002 ± 0.0005,IsActiveMember
0 ± 0.0000,HasCrCard
0 ± 0.0000,Tenure
-0.0000 ± 0.0002,EstimatedSalary


In [16]:
X_train_new, X_test_new, y_train_new, y_test_new =  train_test_split(X, y, test_size = 0.30, random_state = 42)
clf = MLPClassifier(random_state = 1, max_iter = 100).fit(X_train_new, y_train_new)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [17]:
clf.score(X_train_new, y_train_new)

0.755

In [18]:
clf.score(X_test_new, y_test_new)

0.761

In [19]:
perm = PermutationImportance(clf, random_state = 42, n_iter = 10).fit(X, y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

Weight,Feature
0.0103 ± 0.0033,Balance
0.0045 ± 0.0076,EstimatedSalary
0.0011 ± 0.0010,Age
0.0005 ± 0.0006,IsActiveMember
0.0000 ± 0.0003,Tenure
0 ± 0.0000,HasCrCard
0 ± 0.0000,NumOfProducts
-0.0000 ± 0.0003,Gender
-0.0000 ± 0.0003,Geography
-0.0002 ± 0.0010,CreditScore


In [23]:
model = keras.Sequential([
    keras.layers.Dense(10, input_shape = (10,), activation = 'relu'),
    keras.layers.Dense(25, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.fit(X_train, y_train, epochs = 50)

Epoch 1/50
219/219 [==============================] - 1s 1ms/step - loss: 788.2372 - accuracy: 0.6877
Epoch 2/50
219/219 [==============================] - 0s 1ms/step - loss: 48.4147 - accuracy: 0.6514
Epoch 3/50
219/219 [==============================] - 0s 1ms/step - loss: 28.4461 - accuracy: 0.6798
Epoch 4/50
219/219 [==============================] - 0s 1ms/step - loss: 38.1641 - accuracy: 0.6764
Epoch 5/50
219/219 [==============================] - 0s 1ms/step - loss: 27.9613 - accuracy: 0.6672
Epoch 6/50
219/219 [==============================] - 0s 1ms/step - loss: 38.3105 - accuracy: 0.6608
Epoch 7/50
219/219 [==============================] - 0s 1ms/step - loss: 36.8570 - accuracy: 0.6935
Epoch 8/50
219/219 [==============================] - 0s 1ms/step - loss: 33.5627 - accuracy: 0.6826
Epoch 9/50
219/219 [==============================] - 0s 1ms/step - loss: 30.0843 - accuracy: 0.6897
Epoch 10/50
219/219 [==============================] - 0s 1ms/step - loss: 30.1327 - accur

In [24]:
model.evaluate(X_test, y_test)

94/94 [==============================] - 0s 863us/step - loss: 6.7290 - accuracy: 0.7850


[6.728978633880615, 0.7850000262260437]

In [26]:
yp = model.predict(X_test)
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.96      0.88      2416
           1       0.23      0.04      0.07       584

    accuracy                           0.79      3000
   macro avg       0.52      0.50      0.48      3000
weighted avg       0.69      0.79      0.72      3000

